# Easy to read pandas: pandas piping (also known as method chaining)

## R pipe

Tidyverse in R has the "pipe" operator %>% which is extremely useful in writing
readable code.

Ex:

``` R
df >%>
    do_first_thing %>%
    do_second_thing %>%
    final_thing
```

The alternative is usually nesting of functions.

Ex:

``` R
final_thing(
    do_second_thing(
        do_first_thing(df)
    )
)
```

Or saving each intermediate step as its own variable.

Ex:

```R
df1 = do_first_thing(df)
df2 = do_second_thing(df1)
df3 = do_final_thing(df2)
```

## Python pipe?

Pandas does not really have something as smooth as tidyverse, unfortunately.

In pandas, there are 2 ways to pipe functions on DataFrames.
1. Method chaining (or dot chaining) (general python)
2. The pipe function (pandas specific)

We have been using 1 for some time now, but it a limited way.
For example:

``` python
mean_runtime_per_class_section = /
    runtimes.groupby(class_section).mean().sort_values(ascending=False)
```

This is an example of method chaining as we are using methods back to back.
We can simply format it in a more readable way by wrapping the method chain 
in parentheses and adding a new line where we want.

``` python
mean_runtime_per_class_section = (
    runtimes
    .groupby(class_section)
    .mean()
    .sort_values(ascending=False)
)
```

If you find your method chain getting too long, consider breaking up into multiple lines as shown above.

Functionally, the results from using "piped" vs not "piped" code are exactly the same. 
However, piping can sometimes make the code easier to read and understand.
Writing code that is easy to understand is an important skill for anyone who shares code with other people.

Furthermore, method chaining is concept that is used across the Python ecosystem.

## Data Science Salary Dataset

Take the following code as an example.
We are exploring a data science salary dataset.

In [1]:
import numpy as np
import pandas as pd

In [2]:
ds_salaries = pd.read_csv("ds_salaries.csv", index_col=0)
ds_salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


## Question

Which U.S., full time, entry level role pays the most for fully remote jobs?

In [3]:
print(ds_salaries.experience_level.value_counts())
print(ds_salaries.remote_ratio.value_counts())
print(ds_salaries.employment_type.value_counts())

SE    280
MI    213
EN     88
EX     26
Name: experience_level, dtype: int64
100    381
0      127
50      99
Name: remote_ratio, dtype: int64
FT    588
PT     10
CT      5
FL      4
Name: employment_type, dtype: int64


### Pipe version

In [4]:
entry_full_time_remote = (
    ds_salaries
    .query(                                        # filter on conditions
        "employment_type == 'FT'"
        "and experience_level == 'EN'"
        "and remote_ratio == 100"
        "and employee_residence == 'US'")
    .filter(items=["job_title", "salary_in_usd"])  # filter columns
    .groupby("job_title")                          # for each job_title
    .agg(["mean", "count"])                        # calculate mean and count
    .sort_values(("salary_in_usd", "mean"), ascending=False)
)

### Intermediate steps

In [5]:
s1 = ds_salaries[
    (ds_salaries.employment_type == "FT") 
    & (ds_salaries.experience_level =="EN")
    & (ds_salaries.remote_ratio == 100)
    & (ds_salaries.employee_residence == "US")
]
s2 = s1[["job_title", "salary_in_usd"]]
s3 = s2.groupby("job_title")
s4 = s3.agg(["mean", "count"])
s5 = s4.sort_values(("salary_in_usd", "mean"), ascending=False)

### Nested

In [6]:
pd.DataFrame.sort_values(
    pd.DataFrame.groupby(
        ds_salaries[
            (ds_salaries.employment_type == "FT") 
            & (ds_salaries.experience_level =="EN")
            & (ds_salaries.remote_ratio == 100)
            & (ds_salaries.employee_residence == "US")]
        [["job_title", "salary_in_usd"]],
        "job_title"
    ).agg(["mean", "count"]),
    by=("salary_in_usd", "mean"),
    ascending=False
)

salary_in_usd      
                                            mean count
job_title                                             
Machine Learning Scientist         225000.000000     1
Machine Learning Engineer          131500.000000     2
Research Scientist                 120000.000000     1
Data Scientist                      93750.000000     4
Data Science Consultant             90000.000000     1
Data Engineer                       84375.000000     4
Data Analyst                        73833.333333     6
Big Data Engineer                   70000.000000     1
Computer Vision Software Engineer   70000.000000     1

The first and second are more readable than the third.
The difference between the first and second are smaller, but I think the first is slightly easier to read.
Could be personal preference.

## Outline

The rest of the lecture is as follows:
1. Overview of methods vs functions
2. Method/dot chaining
3. Useful DataFrame/Series methods
4. Pandas pipe function

Before we dive specific examples of useful methods for chaining, let's first briefly talk about methods in general.

## Methods vs Functions

Methods are just functions that are "attached" to a specific class.
For example, the *numpy library* has the sum **function**:

``` python
    np.sum
```

which can be used on things like numpy arrays.

In [7]:
X = np.array([1, 2, 3, 4])
np.sum(X)

10

But all *numpy arrays*, like **X** in the above cell, also have their own sum **method**.
To use it you simply type ".sum()" at the end of a numpy array.
This will call the "sum()" function specific to the array class.

In [8]:
print(X.sum())

10


In the previous example, *np.sum* is a **function**, while the *.sum()* after
a numpy array is a **method** that is "attached" to all numpy ndarrays
(the numpy arrays we've been working with).
Specifically, that ".sum()" method is attached to the array class like

``` python
np.ndarray.sum()
```

So anything that is a numpy array type will be able to use the ".sum()" method.

In the pandas library, we've frequently used the ".unique()" method of pandas Series.

``` python
user_names_series.unique()
```

Pandas Series and DataFrames have many useful methods that we've been using throughout the semester.

### Dot Notation

All methods follow the same "dot notation" even for objects and classes that
are not numpy arrays.
For example, lists have an *append* **method** which adds a new value to the end of
a list.
Using it looks like:

In [9]:
xlist = [] # initialize and empty list
xlist.append(1) # append a 1 to the list
print(xlist)

xlist.append("asdfasdf")
print(xlist)

[1]
[1, 'asdfasdf']


## Method Chaining

Now, a really cool thing happens when a method returns an object that is the
same type or class as the original object.

It allows you to use the "dot notation" repeatedly and create a pipeline of sorts.
This is super useful in pandas, because lots of DataFrame/Series methods return Series/DataFrame themselves.
Because these methods return a Series/DataFrame you can immediately chain another Series/DataFrame method.
This looks like

``` python
    df1.method1() -> returns a dataframe, say df2
    df2.method2() -> returns a dataframe, say df3
    df3.method3() -> etc
```

In code, it will look like this:

``` python
    df1.method1().method2().method3().etc
```

This is the core idea of "Method Chaining" (also called "dot chaining").
But as the method chain gets longer, it become harder to read, so you can 
organize them by wrapping them in parentheses.

``` python
    result = (
        df
        .method1()
        .method2()
        .method3()
        .etc()
    )
```

So, what methods are available for DataFrames?
Here are some simple useful ones.

### Some useful DataFrame methods

* query
* filter
* assign
* rename
* drop

but there are many more.
Check out https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html for a
more exhaustive list.
Pandas series methods (including str methods) can also be found https://pandas.pydata.org/pandas-docs/stable/reference/series.html
But we'll stick with these few for now.

#### [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html#pandas.DataFrame.query)

Select rows of a DataFrame based on certain conditional statements. Using boolean masks or the *.loc* method can achieve similar results.

#### [filter](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.filter.html#pandas.DataFrame.filter)

Select certain columns of a DataFrame, or certain indices.

#### [assign](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html#pandas.DataFrame.assign)

Add a new column to a DataFrame, or replace an existing one.

#### [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html#pandas.DataFrame.rename)

Rename columns of a DataFrame or indices.

#### [drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html#pandas.DataFrame.drop)

Drop certain columns of a DataFrame or indices.

### query

In [10]:
df = pd.read_csv("football.csv")

Two ways to do the same thing.

In [11]:
df.query("Quarter == 5").head()
df[df["Quarter"] == 5].head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
1637,2020120604,12/6/20,5,1,49,NaN,NaN,0,0,100,...,0,0,NaN,0,OPP,0,NaN,0,NaN,0
1638,2020120604,12/6/20,5,1,53,NaN,NaN,0,0,100,...,0,0,NaN,0,OPP,0,NaN,0,NaN,0
1639,2020120604,12/6/20,5,2,0,MIN,JAX,2,2,98,...,0,0,CENTER,2,OPP,0,NaN,0,NaN,0
1922,2020120604,12/6/20,5,3,28,MIN,JAX,2,6,86,...,0,0,RIGHT END,14,OPP,0,NaN,0,NaN,0
1923,2020120604,12/6/20,5,4,3,MIN,JAX,1,10,82,...,0,0,CENTER,18,OPP,0,NaN,0,NaN,0


The two lines achieve the same thing, but one could argue that the first is more readable than the second.
According to the docs, using query can be slower for small datasets but faster for large ones.
This difference is probably negligible in most cases.

If a column name has a space, then wrap it in backticks.
Strings in the query can use single or double quotes, just use the opposite of the one it is wrapped in.

In [12]:
df["Play Type"] = df["PlayType"] # add a new column with a space in the name
df.query("`Play Type` == 'KICK OFF'").head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Play Type
5202,2020092708,9/27/20,4,15,0,IND,NYJ,0,0,100,...,0,NaN,0,OPP,0,NaN,0,NaN,0,KICK OFF
5971,2020091301,9/13/20,2,0,6,BAL,CLE,0,0,35,...,0,NaN,35,OWN,0,NaN,0,NaN,0,KICK OFF
6081,2020091000,9/10/20,4,2,38,HOU,KC,0,0,35,...,0,NaN,35,OWN,0,NaN,0,NaN,0,KICK OFF
6082,2020091300,9/13/20,1,11,6,ATL,SEA,0,0,35,...,0,NaN,35,OWN,0,NaN,0,NaN,0,KICK OFF
6097,2020091000,9/10/20,1,15,0,KC,HOU,0,0,35,...,0,NaN,35,OWN,0,NaN,0,NaN,0,KICK OFF


The query method can also contain multiple conditional statements.
The following two lines are equivalent.

In [13]:
df.query("IsPass == True and IsIncomplete == False and IsInterception == False").head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Play Type
3,2020122600,12/26/20,2,13,48,TB,DET,2,6,53,...,0,NaN,47,OPP,0,NaN,0,NaN,0,PASS
4,2020122600,12/26/20,2,13,24,TB,DET,1,10,65,...,0,NaN,35,OPP,0,NaN,0,NaN,0,PASS
5,2020122601,12/26/20,1,13,53,ARI,SF,1,10,61,...,0,NaN,39,OPP,0,NaN,0,NaN,0,PASS
8,2020122602,12/26/20,3,10,38,MIA,LV,2,8,90,...,0,NaN,10,OPP,0,NaN,0,NaN,0,PASS
10,2020122602,12/26/20,4,5,33,MIA,LV,1,10,65,...,0,NaN,35,OPP,0,NaN,0,NaN,0,PASS


In [14]:
df[
    (df.IsPass == True) &
    (df.IsIncomplete == False) &
    (df.IsInterception == False)
].head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Play Type
3,2020122600,12/26/20,2,13,48,TB,DET,2,6,53,...,0,NaN,47,OPP,0,NaN,0,NaN,0,PASS
4,2020122600,12/26/20,2,13,24,TB,DET,1,10,65,...,0,NaN,35,OPP,0,NaN,0,NaN,0,PASS
5,2020122601,12/26/20,1,13,53,ARI,SF,1,10,61,...,0,NaN,39,OPP,0,NaN,0,NaN,0,PASS
8,2020122602,12/26/20,3,10,38,MIA,LV,2,8,90,...,0,NaN,10,OPP,0,NaN,0,NaN,0,PASS
10,2020122602,12/26/20,4,5,33,MIA,LV,1,10,65,...,0,NaN,35,OPP,0,NaN,0,NaN,0,PASS


Variables can be reference in the query string using @

In [15]:
kicks = ["FIELD GOAL", "PUNT", "EXTRA POINT", "KICK OFF"]
df.query("PlayType in @kicks").head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Play Type
17,2020122705,12/27/20,2,1,11,NYG,BAL,4,6,87,...,0,NaN,13,OPP,0,NaN,0,NaN,0,FIELD GOAL
24,2020122706,12/27/20,2,7,46,HOU,CIN,4,5,58,...,0,NaN,42,OPP,0,NaN,0,NaN,0,PUNT
71,2021010311,1/3/21,2,4,40,LV,DEN,4,8,58,...,0,NaN,42,OPP,0,NaN,0,NaN,0,PUNT
78,2021010311,1/3/21,1,0,17,DEN,LV,0,0,85,...,0,NaN,15,OPP,0,NaN,0,NaN,0,EXTRA POINT
90,2021010313,1/3/21,4,6,0,IND,JAX,4,11,55,...,0,NaN,45,OPP,0,NaN,0,NaN,0,PUNT


In [16]:
df[df.PlayType.isin(kicks)].head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Play Type
17,2020122705,12/27/20,2,1,11,NYG,BAL,4,6,87,...,0,NaN,13,OPP,0,NaN,0,NaN,0,FIELD GOAL
24,2020122706,12/27/20,2,7,46,HOU,CIN,4,5,58,...,0,NaN,42,OPP,0,NaN,0,NaN,0,PUNT
71,2021010311,1/3/21,2,4,40,LV,DEN,4,8,58,...,0,NaN,42,OPP,0,NaN,0,NaN,0,PUNT
78,2021010311,1/3/21,1,0,17,DEN,LV,0,0,85,...,0,NaN,15,OPP,0,NaN,0,NaN,0,EXTRA POINT
90,2021010313,1/3/21,4,6,0,IND,JAX,4,11,55,...,0,NaN,45,OPP,0,NaN,0,NaN,0,PUNT


## Example

Do some quarters have better passing than other quarters?

In [17]:
passes_per_quarter = (
    df[["IsPass", "Quarter"]]
    .query("IsPass == True")
    .groupby("Quarter")
    .count()
)
successful_passes_per_quarter = (
    df[["IsPass", "Quarter", "IsIncomplete", "IsInterception"]]
    .query("IsPass == True and IsIncomplete == False and IsInterception == False")
    .groupby("Quarter")
    [["IsPass"]]
    .count()
)
(
    (successful_passes_per_quarter / passes_per_quarter)
    .reset_index()
    .rename(columns={"IsPass": "SuccessfulPass"})
)

,Quarter,SuccessfulPass
0,1,0.665522
1,2,0.654946
2,3,0.660534
3,4,0.644925
4,5,0.565217


### filter

Select specific columns or labels.
Similar to loc.

In [18]:
df.head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Play Type
0,2020122602,12/26/20,2,9,44,LV,MIA,2,10,82,...,0,NaN,18,OPP,0,NaN,0,NaN,0,SACK
1,2020122602,12/26/20,2,9,3,LV,MIA,3,19,73,...,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0,RUSH
2,2020122500,12/25/20,3,1,6,MIN,NO,1,1,99,...,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0,RUSH
3,2020122600,12/26/20,2,13,48,TB,DET,2,6,53,...,0,NaN,47,OPP,0,NaN,0,NaN,0,PASS
4,2020122600,12/26/20,2,13,24,TB,DET,1,10,65,...,0,NaN,35,OPP,0,NaN,0,NaN,0,PASS


In [19]:
df.filter(items=["OffenseTeam", "DefenseTeam", "YardLine", "PlayType"])

,OffenseTeam,DefenseTeam,YardLine,PlayType
0,LV,MIA,82,SACK
1,LV,MIA,73,RUSH
2,MIN,NO,99,RUSH
3,TB,DET,53,PASS
4,TB,DET,65,PASS
...,...,...,...,...
25853,ATL,SEA,82,PASS
25854,ATL,SEA,80,PASS
25855,NaN,NaN,100,TIMEOUT
25856,ATL,SEA,79,PASS


In [20]:
df.filter(like="Team")

,OffenseTeam,DefenseTeam,TeamWin,PenaltyTeam
0,LV,MIA,0,NaN
1,LV,MIA,0,NaN
2,MIN,NO,0,NaN
3,TB,DET,0,NaN
4,TB,DET,0,NaN
...,...,...,...,...
25853,ATL,SEA,0,NaN
25854,ATL,SEA,0,NaN
25855,NaN,NaN,0,NaN
25856,ATL,SEA,0,NaN


### assign

Create new columns, overwrite existing ones.

In [21]:
ds_salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [22]:
(
    ds_salaries
    .assign(in_office_ratio = 100 - ds_salaries.remote_ratio)
    .filter(like="ratio") # select columns that contain 'ratio'
)

,remote_ratio,in_office_ratio
0,0,100
1,0,100
2,50,50
3,0,100
4,50,50
...,...,...
602,100,0
603,100,0
604,0,100
605,100,0


### rename

Renames columns or labels.

We can rename specific columns.

In [23]:
df.rename(columns={"OffenseTeam": "offense_team"})

,GameId,GameDate,Quarter,Minute,Second,offense_team,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Play Type
0,2020122602,12/26/20,2,9,44,LV,MIA,2,10,82,...,0,NaN,18,OPP,0,NaN,0,NaN,0,SACK
1,2020122602,12/26/20,2,9,3,LV,MIA,3,19,73,...,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0,RUSH
2,2020122500,12/25/20,3,1,6,MIN,NO,1,1,99,...,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0,RUSH
3,2020122600,12/26/20,2,13,48,TB,DET,2,6,53,...,0,NaN,47,OPP,0,NaN,0,NaN,0,PASS
4,2020122600,12/26/20,2,13,24,TB,DET,1,10,65,...,0,NaN,35,OPP,0,NaN,0,NaN,0,PASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25853,2020091300,9/13/20,4,1,17,ATL,SEA,3,7,82,...,0,NaN,18,OPP,0,NaN,0,NaN,0,PASS
25854,2020091300,9/13/20,4,1,38,ATL,SEA,2,9,80,...,0,NaN,20,OPP,0,NaN,0,NaN,0,PASS
25855,2020091300,9/13/20,4,5,18,NaN,NaN,0,0,100,...,0,NaN,0,OPP,0,NaN,0,NaN,0,TIMEOUT
25856,2020091300,9/13/20,4,2,0,ATL,SEA,1,10,79,...,0,NaN,21,OPP,0,NaN,0,NaN,0,PASS


Rename all columns with a function.

In [24]:
df.rename(mapper=str.lower, axis=1)

,gameid,gamedate,quarter,minute,second,offenseteam,defenseteam,down,togo,yardline,...,istwopointconversionsuccessful,rushdirection,yardlinefixed,yardlinedirection,ispenaltyaccepted,penaltyteam,isnoplay,penaltytype,penaltyyards,play type
0,2020122602,12/26/20,2,9,44,LV,MIA,2,10,82,...,0,NaN,18,OPP,0,NaN,0,NaN,0,SACK
1,2020122602,12/26/20,2,9,3,LV,MIA,3,19,73,...,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0,RUSH
2,2020122500,12/25/20,3,1,6,MIN,NO,1,1,99,...,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0,RUSH
3,2020122600,12/26/20,2,13,48,TB,DET,2,6,53,...,0,NaN,47,OPP,0,NaN,0,NaN,0,PASS
4,2020122600,12/26/20,2,13,24,TB,DET,1,10,65,...,0,NaN,35,OPP,0,NaN,0,NaN,0,PASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25853,2020091300,9/13/20,4,1,17,ATL,SEA,3,7,82,...,0,NaN,18,OPP,0,NaN,0,NaN,0,PASS
25854,2020091300,9/13/20,4,1,38,ATL,SEA,2,9,80,...,0,NaN,20,OPP,0,NaN,0,NaN,0,PASS
25855,2020091300,9/13/20,4,5,18,NaN,NaN,0,0,100,...,0,NaN,0,OPP,0,NaN,0,NaN,0,TIMEOUT
25856,2020091300,9/13/20,4,2,0,ATL,SEA,1,10,79,...,0,NaN,21,OPP,0,NaN,0,NaN,0,PASS


Rename all labels with a function.

In [25]:
df.rename(mapper=lambda x: x + 4) # default is axis=0, the index so we don't need to specify

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Play Type
4,2020122602,12/26/20,2,9,44,LV,MIA,2,10,82,...,0,NaN,18,OPP,0,NaN,0,NaN,0,SACK
5,2020122602,12/26/20,2,9,3,LV,MIA,3,19,73,...,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0,RUSH
6,2020122500,12/25/20,3,1,6,MIN,NO,1,1,99,...,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0,RUSH
7,2020122600,12/26/20,2,13,48,TB,DET,2,6,53,...,0,NaN,47,OPP,0,NaN,0,NaN,0,PASS
8,2020122600,12/26/20,2,13,24,TB,DET,1,10,65,...,0,NaN,35,OPP,0,NaN,0,NaN,0,PASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25857,2020091300,9/13/20,4,1,17,ATL,SEA,3,7,82,...,0,NaN,18,OPP,0,NaN,0,NaN,0,PASS
25858,2020091300,9/13/20,4,1,38,ATL,SEA,2,9,80,...,0,NaN,20,OPP,0,NaN,0,NaN,0,PASS
25859,2020091300,9/13/20,4,5,18,NaN,NaN,0,0,100,...,0,NaN,0,OPP,0,NaN,0,NaN,0,TIMEOUT
25860,2020091300,9/13/20,4,2,0,ATL,SEA,1,10,79,...,0,NaN,21,OPP,0,NaN,0,NaN,0,PASS


### drop

Drop labels or columns

In [26]:
df.drop(columns=["GameId", "GameDate", "Quarter"])

,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,Unnamed: 10,SeriesFirstDown,Unnamed: 12,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Play Type
0,9,44,LV,MIA,2,10,82,NaN,0,NaN,...,0,NaN,18,OPP,0,NaN,0,NaN,0,SACK
1,9,3,LV,MIA,3,19,73,NaN,0,NaN,...,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0,RUSH
2,1,6,MIN,NO,1,1,99,NaN,0,NaN,...,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0,RUSH
3,13,48,TB,DET,2,6,53,NaN,1,NaN,...,0,NaN,47,OPP,0,NaN,0,NaN,0,PASS
4,13,24,TB,DET,1,10,65,NaN,1,NaN,...,0,NaN,35,OPP,0,NaN,0,NaN,0,PASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25853,1,17,ATL,SEA,3,7,82,NaN,1,NaN,...,0,NaN,18,OPP,0,NaN,0,NaN,0,PASS
25854,1,38,ATL,SEA,2,9,80,NaN,0,NaN,...,0,NaN,20,OPP,0,NaN,0,NaN,0,PASS
25855,5,18,NaN,NaN,0,0,100,NaN,1,NaN,...,0,NaN,0,OPP,0,NaN,0,NaN,0,TIMEOUT
25856,2,0,ATL,SEA,1,10,79,NaN,0,NaN,...,0,NaN,21,OPP,0,NaN,0,NaN,0,PASS


All the functions an be done in different ways, for example using loc or masks.

## Pipe function

Piping is another way to chain operations on DataFrames.
It's a little more clunky than dot chaining, but is more flexible since you could use arbitrary functions (that expect Series or DataFrames).

Again, you can do it all on one line, or wrap over multiple lines with parentheses.

In [27]:
(
    ds_salaries
    .pipe(pd.DataFrame.query, "experience_level == 'EN'")
    .pipe(pd.DataFrame.filter, like="sal")
)

,salary,salary_currency,salary_in_usd
5,72000,USD,72000
10,45000,EUR,51321
12,35000,EUR,39916
16,4450000,JPY,41689
18,423000,INR,5707
...,...,...,...
512,65000,USD,65000
514,20000,USD,20000
521,10000,USD,10000
600,67000,USD,67000


## Conclusion

Method chaining is an easy way to make your code more readable.
It may also help approaching the problem in a systematic linear way.
However, it is not required and ultimately 'readability' up to each user.

One big downside of long method chains is that you are not able to easily debug intermediate steps partway through the chain.
So if you do require testing/checking intermediate steps, consider breaking up the pipeline.
